In [1]:
import pandas as pd
import json

In [2]:
# Ukuran chunk
chunk_size = 1000000  

# Buka ulang iterator setiap kali ingin membacanya
df1_iter = pd.read_json('filtered_meta_Books.json', lines=True, chunksize=chunk_size)

for df1_chunk in df1_iter:
    # Pastikan kolom hanya berisi list sebelum melakukan applymap
    count_empty_list = df1_chunk.applymap(lambda x: isinstance(x, list) and x == []).sum()
    count_comma_list = df1_chunk.applymap(lambda x: isinstance(x, list) and x == [', ']).sum()

    print("Jumlah baris dengan nilai [] di setiap kolom:\n", count_empty_list)
    print("Jumlah baris dengan nilai [, ] di setiap kolom:\n", count_comma_list)
    
    # Hentikan setelah satu chunk (hapus jika ingin menghitung semua)
    break  

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_57852\4135744597.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  count_empty_list = df1_chunk.applymap(lambda x: isinstance(x, list) and x == []).sum()
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_57852\4135744597.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  count_comma_list = df1_chunk.applymap(lambda x: isinstance(x, list) and x == [', ']).sum()


Jumlah baris dengan nilai [] di setiap kolom:
 description    119328
title               0
asin                0
dtype: int64
Jumlah baris dengan nilai [, ] di setiap kolom:
 description    0
title          0
asin           0
dtype: int64


In [4]:
total_empty_list = 0

for df1_chunk in df1_iter:
    total_empty_list += df1_chunk['description'].apply(lambda x: x == []).sum()

print(f"Total baris dengan nilai [] di seluruh dataset: {total_empty_list}")


Total baris dengan nilai [] di seluruh dataset: 0


In [4]:
# Membaca file JSON dalam mode chunk
chunk_size = 1000000  # Sesuaikan dengan kapasitas memori
df1_iter = pd.read_json('filtered_meta_Books.json', lines=True, chunksize=chunk_size)

# File output untuk menyimpan hasil
output_file = 'filtered_meta_Books_cleaned.json'

# Membuka file untuk menulis hasil bersih
with open(output_file, 'w') as f_out:
    for df1_chunk in df1_iter:
        # Filter baris yang tidak kosong di kolom 'description'
        df1_filtered = df1_chunk[df1_chunk['description'].apply(lambda x: x != [])]
        
        # Simpan ke file JSON (append untuk tiap chunk)
        df1_filtered.to_json(f_out, orient='records', lines=True)

print(f"File bersih telah disimpan sebagai '{output_file}'")

File bersih telah disimpan sebagai 'filtered_meta_Books_cleaned.json'


In [5]:
# Ukuran chunk
chunk_size = 1000000  

# Buka ulang iterator setiap kali ingin membacanya
df2_iter = pd.read_json('filtered_Books.json', lines=True, chunksize=chunk_size)



for df2_chunk in df2_iter:
    # Pastikan kolom hanya berisi list sebelum melakukan applymap
    count_empty_list = df2_chunk.applymap(lambda x: isinstance(x, list) and x == []).sum()
    count_comma_list = df2_chunk.applymap(lambda x: isinstance(x, list) and x == [', ']).sum()

    print("Jumlah baris dengan nilai [] di setiap kolom:\n", count_empty_list)
    print("Jumlah baris dengan nilai [, ] di setiap kolom:\n", count_comma_list)
    
    # Hentikan setelah satu chunk (hapus jika ingin menghitung semua)
    break  

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22616\2466272136.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  count_empty_list = df2_chunk.applymap(lambda x: isinstance(x, list) and x == []).sum()
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_22616\2466272136.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  count_comma_list = df2_chunk.applymap(lambda x: isinstance(x, list) and x == [', ']).sum()


Jumlah baris dengan nilai [] di setiap kolom:
 reviewerID    0
asin          0
overall       0
reviewTime    0
dtype: int64
Jumlah baris dengan nilai [, ] di setiap kolom:
 reviewerID    0
asin          0
overall       0
reviewTime    0
dtype: int64


In [5]:
df1_chunk.head()

,description,title,asin
2000000,"[, Meda Ryan was born in West Cork and educate...",Tom Barry: IRA Freedom Fighter,1856354806
2000001,"[, ]",Sarsfield &amp; the Jacobites,1856354083
2000002,[Carla Blake was born in South Africa but has ...,Irish Cookbook,1856355047
2000003,"[<i>The stories that appear are painful, detai...",Melancholy Madness: A Coroner's Casebook,1856354245
2000004,"[, ]",The Book of Irish Limericks,1856352617


In [7]:
df2_chunk.head()

,reviewerID,asin,overall,reviewTime
0,A1C6M8LCIX4M6M,0001713353,5,"08 12, 2005"
1,A1REUF3A1YCPHM,0001713353,5,"03 30, 2005"
2,A1YRBRK2XM5D5,0001713353,5,"04 4, 2004"
3,A1V8ZR5P78P4ZU,0001713353,5,"02 21, 2004"
4,A2ZB06582NXCIV,0001713353,5,"10 3, 2016"


In [8]:
# Membaca file JSON dalam mode chunk untuk efisiensi memori
chunk_size = 1000000  
df1_iter = pd.read_json('filtered_meta_Books_cleaned.json', lines=True, chunksize=chunk_size)
df2_iter = pd.read_json('filtered_Books.json', lines=True, chunksize=chunk_size)

# File output untuk hasil merge
output_file = 'merge_Books.csv'

# Membuka file CSV untuk menulis hasil merge
with open(output_file, 'w') as f_out:
    header_written = False  # Pastikan header hanya ditulis sekali
    
    for df1_chunk, df2_chunk in zip(df1_iter, df2_iter):
        # Menggabungkan berdasarkan 'asin'
        merge_Books = pd.merge(df1_chunk, df2_chunk, on='asin', how='inner')

        # Menyimpan ke file CSV (append untuk tiap chunk)
        merge_Books.to_csv(f_out, mode='a', index=False, header=not header_written)
        header_written = True  # Setelah iterasi pertama, header tidak ditulis lagi

print(f"Hasil merge telah disimpan sebagai '{output_file}'")

Hasil merge telah disimpan sebagai 'merge_Books.csv'


In [3]:
# Membaca file CSV hasil merge
merge_Books = pd.read_csv('merge_Books.csv')

# Menampilkan 5 baris teratas
print(merge_Books.head())

                                         description  \
0  ['This is a collection of classic gospel hymns...   
1  ['This is a collection of classic gospel hymns...   
2  ['This is a collection of classic gospel hymns...   
3  ['This is a collection of classic gospel hymns...   
4  ['This is a collection of classic gospel hymns...   

                                        title        asin      reviewerID  \
0  Heavenly Highway Hymns: Shaped-Note Hymnal  0000013765  A3BED5QFJWK88M   
1  Heavenly Highway Hymns: Shaped-Note Hymnal  0000013765   AYEDW3BFK53XK   
2  Heavenly Highway Hymns: Shaped-Note Hymnal  0000013765  A2EIPZNHAEXZHJ   
3  Heavenly Highway Hymns: Shaped-Note Hymnal  0000013765  A37W6POFWIVG13   
4  Heavenly Highway Hymns: Shaped-Note Hymnal  0000013765  A2SUAM1J3GNN3B   

   overall   reviewTime  
0        4  10 16, 2012  
1        5   01 2, 2012  
2        4  12 28, 2011  
3        5  09 16, 2011  
4        5  09 13, 2009  


In [4]:
# Menghitung jumlah baris
num_rows = merge_Books.shape[0]
print("Jumlah baris:", num_rows)

Jumlah baris: 752034


In [ ]:
from sklearn.preprocessing import LabelEncoder
import base64

# Tokenizing 'reviewerID'
label_encoder_reviewer = LabelEncoder()
merge_Books['reviewerID'] = label_encoder_reviewer.fit_transform(merge_Books['reviewerID'])

# Fungsi untuk mengubah asin menjadi format yang mirip dengan sebelumnya
def encode_asin(asin):
    encoded = base64.b32encode(asin.encode()).decode()[:10]  # Base32 encoding dengan 10 karakter
    return encoded

# Terapkan encoding ke kolom 'asin'
merge_Books['asin'] = merge_Books['asin'].astype(str).apply(encode_asin)

In [19]:
# Menampilkan hasil 5 baris pertama
print(merge_Books.head(100))

                                          description  \
0   ['This is a collection of classic gospel hymns...   
1   ['This is a collection of classic gospel hymns...   
2   ['This is a collection of classic gospel hymns...   
3   ['This is a collection of classic gospel hymns...   
4   ['This is a collection of classic gospel hymns...   
..                                                ...   
95  ['William Shakespeare is widely regarded as th...   
96  ['William Shakespeare is widely regarded as th...   
97  ['William Shakespeare is widely regarded as th...   
98  ['William Shakespeare is widely regarded as th...   
99  ['William Shakespeare is widely regarded as th...   

                                                title        asin  reviewerID  \
0          Heavenly Highway Hymns: Shaped-Note Hymnal  MNTGGZBSGA      366595   
1          Heavenly Highway Hymns: Shaped-Note Hymnal  MNTGGZBSGA      591833   
2          Heavenly Highway Hymns: Shaped-Note Hymnal  MNTGGZBSGA      2

In [20]:
# Menyimpan hasil tokenization ke file baru
merge_Books.to_csv('merge_Books_tokenized(2).csv', index=False)

In [22]:
# Load dataset
df = pd.read_csv('merge_Books_tokenized(2).csv')

# Pilih 2.000 item pertama berdasarkan 'asin' (pastikan unik)
selected_asins = df['asin'].drop_duplicates().head(5)

# Filter dataset untuk hanya menyertakan review dari 2.000 item tersebut
filtered_df = df[df['asin'].isin(selected_asins)]

# Simpan hasil ke dalam file baru
filtered_df.to_csv('Dataset_Books_for_analysis(2).csv', index=False)

# Menampilkan jumlah unique item dan reviewer
num_items = filtered_df['asin'].nunique()
num_reviewers = filtered_df['reviewerID'].nunique()
print(f"Jumlah item unik: {num_items}, Jumlah reviewer unik: {num_reviewers}")

Jumlah item unik: 5, Jumlah reviewer unik: 3611


In [ ]:
# # Load dataset
# df = pd.read_csv('merge_Books_tokenized(2).csv')

# # Menghitung jumlah unique reviewerID untuk setiap ASIN
# reviewer_counts = df.groupby('asin')['reviewerID'].nunique().reset_index()

# # Filter hanya ASIN yang memiliki maksimal 20 reviewer
# filtered_asins = reviewer_counts[reviewer_counts['reviewerID'] <= 20]

# # Ambil 5 ASIN dengan jumlah reviewer terbanyak dalam batasan ini
# top_5_asins = filtered_asins.sort_values(by='reviewerID', ascending=False).head(5)['asin']

# # Filter dataset berdasarkan 5 ASIN tersebut
# filtered_df = df[df['asin'].isin(top_5_asins)]

# # Simpan hasil ke dalam file baru
# filtered_df.to_csv('Dataset_Books_for_analysis(2).csv', index=False)

# # Menampilkan jumlah unique item dan reviewer
# num_items = filtered_df['asin'].nunique()
# num_reviewers = filtered_df['reviewerID'].nunique()
# print(f"Jumlah item unik: {num_items}, Jumlah reviewer unik: {num_reviewers}")

Jumlah item unik: 5, Jumlah reviewer unik: 100


In [5]:
# Load dataset
merge_Books = pd.read_csv('merge_Books_tokenized(2).csv')

# Pilih atribut tertentu (sesuaikan dengan kebutuhan)
selected_columns = ['reviewerID', 'asin', 'title']
filtered_df = merge_Books[selected_columns]

# Simpan hasil ke dalam file baru
output_file = 'Attributes_Books.csv'
filtered_df.to_csv(output_file, index=False)

print(f"File dengan atribut terpilih telah disimpan sebagai '{output_file}'")


File dengan atribut terpilih telah disimpan sebagai 'Attributes_Books.csv'


In [6]:
# Path untuk file input dan output
csv_path = 'Attributes_Books.csv'  # Sesuaikan dengan lokasi file
json_path = 'Attributes_Books.json'  # Path output JSONL

# Membaca CSV
df = pd.read_csv(csv_path)

# Menyimpan sebagai JSONL (JSON Lines)
df.to_json(json_path, orient="records", lines=True, force_ascii=False)

print(f"✅ File berhasil dikonversi: {json_path}")

✅ File berhasil dikonversi: Attributes_Books.json


In [7]:
import json
import os

# Path file JSON (sesuaikan dengan lokasi file di lokal)
json_path = "Attributes_Books.json"  # File input JSONL
output_json_path = "Attributes_Books_array.json"  # Output JSON array

# Cek apakah file JSONL ada
if not os.path.exists(json_path):
    print(f"❌ File tidak ditemukan: {json_path}")
else:
    json_array = []

    # Membaca file JSONL dan mengonversi ke format array
    with open(json_path, "r", encoding="utf-8") as f:
        for line in f:
            try:
                json_array.append(json.loads(line.strip()))
            except json.JSONDecodeError as e:
                print(f"⚠️ Error decoding JSON: {e}")

    # Menyimpan hasil dalam bentuk array ke file baru
    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(json_array, f, indent=4, ensure_ascii=False)

    print(f"✅ File JSON dalam bentuk array telah disimpan ke {output_json_path}")


✅ File JSON dalam bentuk array telah disimpan ke Attributes_Books_array.json


In [28]:
# Load dataset
df = pd.read_csv('merge_Books_tokenized(2).csv')

# Pilih 2.000 item pertama berdasarkan 'asin' (pastikan unik)
selected_asins = df['reviewerID'].drop_duplicates().head(5)

# Filter dataset untuk hanya menyertakan review dari 2.000 item tersebut
filtered_df = df[df['reviewerID'].isin(selected_asins)]

# Simpan hasil ke dalam file baru
filtered_df.to_csv('Dataset_Books_for_analysis_reviewerID.csv', index=False)

# Menampilkan jumlah unique item dan reviewer
num_items = filtered_df['asin'].nunique()
num_reviewers = filtered_df['reviewerID'].nunique()
print(f"Jumlah item unik: {num_items}, Jumlah reviewer unik: {num_reviewers}")

Jumlah item unik: 1, Jumlah reviewer unik: 5


In [29]:
# Load dataset
df_analysis = pd.read_csv('Dataset_Books_for_analysis_reviewerID.csv')

# Pilih atribut tertentu (sesuaikan dengan kebutuhan)
selected_columns = ['reviewerID', 'asin', 'title']
filtered_df = df_analysis[selected_columns]

# Simpan hasil ke dalam file baru
output_file = 'selected_attributes_Books_reviewerID.csv'
filtered_df.to_csv(output_file, index=False)

print(f"File dengan atribut terpilih telah disimpan sebagai '{output_file}'")


File dengan atribut terpilih telah disimpan sebagai 'selected_attributes_Books_reviewerID.csv'


In [32]:
# Path untuk file input dan output
csv_path = 'selected_attributes_Books_reviewerID.csv'  # Sesuaikan dengan lokasi file
json_path = 'selected_attributes_Books_reviewerID.json'  # Path output JSONL

# Membaca CSV
df = pd.read_csv(csv_path)

# Menyimpan sebagai JSONL (JSON Lines)
df.to_json(json_path, orient="records", lines=True, force_ascii=False)

print(f"✅ File berhasil dikonversi: {json_path}")

✅ File berhasil dikonversi: selected_attributes_Books_reviewerID.json


In [33]:
import json
import os

# Path file JSON (sesuaikan dengan lokasi file di lokal)
json_path = "selected_attributes_Books_reviewerID.json"  # File input JSONL
output_json_path = "selected_attributes_Books_array_reviewerID.json"  # Output JSON array

# Cek apakah file JSONL ada
if not os.path.exists(json_path):
    print(f"❌ File tidak ditemukan: {json_path}")
else:
    json_array = []

    # Membaca file JSONL dan mengonversi ke format array
    with open(json_path, "r", encoding="utf-8") as f:
        for line in f:
            try:
                json_array.append(json.loads(line.strip()))
            except json.JSONDecodeError as e:
                print(f"⚠️ Error decoding JSON: {e}")

    # Menyimpan hasil dalam bentuk array ke file baru
    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(json_array, f, indent=4, ensure_ascii=False)

    print(f"✅ File JSON dalam bentuk array telah disimpan ke {output_json_path}")


✅ File JSON dalam bentuk array telah disimpan ke selected_attributes_Books_array_reviewerID.json
